In [1]:
from model import *

In [2]:
model = MyModel()

if torch.cuda.is_available():
    device = torch.device('cuda')  # Default CUDA device
    num_gpus = torch.cuda.device_count()
    print(f'{num_gpus} GPUs available')

    model = nn.DataParallel(model)  # Use multiple GPUs
    model.to(device)
else:
    print('CUDA is not available.')

model.eval() 

1 GPUs available


DataParallel(
  (module): MyModel(
    (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), dilation=(2, 2))
    (conv2): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), dilation=(4, 4))
    (conv3): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1), dilation=(16, 16))
    (den_inp): Conv2d(64, 3, kernel_size=(1, 1), stride=(1, 1))
    (den): Den(
      (conv1): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv3): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (actc): ACTC(
        (conv1): Conv2d(64, 7, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(64, 7, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
        (conv3): Conv2d(64, 7, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
        (conv4): Conv2d(21, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
      (mdr_inp): Conv2d(64, 3, kernel_size=(1, 1), stride=(1, 

In [3]:
def get_transform():
    return Compose([
        ToTensor(),
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

trans = get_transform()

In [4]:
def tensor_to_image(tensor):
    tensor = tensor.cuda()

    # Define the mean and std
    mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
    std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)

    # Unnormalize the tensor
    std = std.cuda()
    mean = mean.cuda()
    tensor = tensor * std + mean

    # Clamp the values in the tensor to the range [0, 1]
    tensor = torch.clamp(tensor, 0, 1)

    # Convert the tensor to a PIL Image and then convert it to a numpy array
    image = transforms.ToPILImage()(tensor).convert("RGB")
    
    return image

In [5]:
# Read the video file
cap = cv2.VideoCapture('/mnt/e/codes_works/super_resolution/videos/input.mp4')

# Create a Pillow Image object to store each frame
im = None
frames = list()

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    # If the frame is empty, break the loop
    if not ret:
        break

    # Convert the frame to Pillow format
    im = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    test = trans(im)
    test = test.unsqueeze(0)
    test = test.cuda()

    out = model(test)

    img = tensor_to_image(out[0])

    frames.append(img)

In [ ]:
# Get the width and height of each frame
widths = []
heights = []
for i, frame in enumerate(frames):
    widths.append(frame.size[0])
    heights.append(frame.size[1])

# Use the largest width and height as the resolution of the output video
width = max(widths)
height = max(heights)
fps = int(cap.get(5))

cap.release()

out = cv2.VideoWriter('/mnt/e/codes_works/super_resolution/videos/output.mp4', cv2.VideoWriter_fourcc(*'XVID'), fps, (width, height))

# Write the frames to the video file
for i, frame in enumerate(frames):
    out.write(frame)

# Release the VideoWriter object
out.release()